<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

**Recuerda que al leer este documento directamente desde GitHub, no estás aprovechando su contenido al 100%.
Editando el código puedes aprender más que leyéndolo. Te recomendamos descargarlo y ejecutarlo desde tu computador para que puedas hacer modificaciones e interactuar con el código.**

# Networking con Python

La programación en redes busca resolver el problema de cómo comunicar distintos computadores, sin importar dónde se encuentren físicamente. Internet nos provee de una infraestructura que nos permite la comunicación entre máquinas a lo largo de la red. En este capítulo exploraremos los conceptos necesarios para conocer los protocolos de comunicación entre computadores, de tal forma de entender cómo podemos enviar y recibir información en forma remota.

## Cómo identificamos las máquinas dentro de internet

Todas las máquinas conectadas a internet tienen un hostname y una dirección IP asociada, por ejemplo, www.python.org tiene la dirección ip 199.27.76.223. Una forma rápida de obtener el ip de una dirección web o hostname es escribiendo "ping $\textit{hostname}$" en la consola (Ej. ping www.python.org). Una dirección IP (Internet Protocol) en la versión 4 (IPv4) corresponde a un número binario de 32 bits (separados en 4 bytes), así en IPv4 pueden existir como máximo $(2^8)^4 = 256^4 = 4.294.967.296$ direcciones ip. Dado que ese número ya nos quedó pequeño para todas las direcciones ip que necesitamos manejar en el mundo, es que apareció IPv6, donde cada dirección tiene 128 bits, dividida en 8 grupos de 16 bits cada uno, representados en notación hexadecimal, es decir, 4 dígitos hexadecimales por cada grupo ($16^4$ valores posibles), separados por dos puntos, ejemplo: 20f1:0db8:0aab:12f1:0110:1bde:0bfd:0001


<h2>Puertos</h2>

Dado que además de comunicarnos con una máquina, en general necesitamos comunicarnos directamente con algún programa, aplicación o servicio dentro de la máquina, necesitamos además de la dirección IP el <b>puerto</b> dentro de la máquina con el cual nos comunicaremos. Por ejemplo, si necesitamos conectarnos con un servidor a través de ftp para transferencia de archivos, debemos conectarnos al servidor a través del puerto 21. Para varias aplicaciones existen números estándar de puertos, aquí algunos ejemplos:

| Puerto | descripción |
|--------|-------------|
| 21     | FTP CONTROL |
| 22     | SSH         | 
| 23     | Telnet      |
| 25     | SMTP (Mail) |
| 37	 |  Time       |
| 42	 | Host Name Server (Nameserv) |
| 53	 | Domain Name System (DNS) |
| 80     | HTTP (Web)  |
| 110    | POP3 (Mail) | 
| 118	 | SQL Services|
| 119    | NNTP (News) | 
| 443    | HTTPS (web) | 


El número de puerto se representa por un número binario de 16 bits, existiendo entonces $2^{16} = 65536$ puertos posibles. Existen tres rangos definidos dentro de la lista de puertos posibles: los puertos conocidos (well-known ports) van en el rango [0-1023], los puertos registrados (the registered ports) que van en el rango  [1024-49151] y los puertos dinámicos o privados (the dynamic or private ports) que van en el rango [49152-65535]. La organización IANA (Internet Assigned Numbers Authority) es responsable de designar y mantener los números de puertos para los dos primeros rangos, el tercer rango en general es usado por el sistema operativo para la asignación de puertos requeridos por distintos programas. 

Cada programa que se comunica dentro de una red debe estar representado entonces por un host y un puerto, en Python por ejemplo representamos el par como una tupla: ("www.yahoo.es", 80) ó ("74.6.50.150", 443). En general hay dos formas de conexión entre puntos en una red: TCP (Transmission Control Protocol) y UDP (User Datagram Protocol). 

**TCP**: En este tipo de conexión se garantiza que los datos van a llegar intactos, sin ningún tipo de pérdida de información, ningún duplicado o cambio en el orden de los datos, obviamente a menos que la conexión falle. En casos de pérdidas de información o falla en las conexiones los paquetes de datos son retransmitidos hasta que llegan satisfactoriamente. Cada paquete TCP lleva asociado una secuencia de números de tal forma que el sistema que recibe los paquetes tiene la información necesaria para re-ensamblarlos en el orden correcto. Además el sistema que recibe los paquetes puede darse cuenta de que falta algún paquete intermedio usando la misma secuencia de números, así cuando falta un paquete el sistema lo solicita de nuevo para su re-transmisión. Algunos ejemplos son: transmisión de archivos vía ftp, envío de correos (SMTP), HTTP, POP3, etc. La siguiente figura (obtenida desde http://software-engineer-training.com/transmission-control-protocol-tcp/) muestra cómo se compone el header de un datagrama TCP:

![](imgs/tcp_header.png)

**UDP**: Permite el envío de datos sin la necesidad de establecer una conexión. Los paquetes de datos enviados a través del protocolo UDP (datagramas) contienen un encabezado suficiente para ser identificados y direccionados correctamente a través de la red. Algunos ejemplos son: streaming de video y audio, información del clima, juegos online, etc. La figura siguiente (obtenida desde http://software-engineer-training.com/tag/udp/) muestra el header de un datagrama UDP:

![](imgs/udp_header.png)

## Sockets:
    
Cuando escribimos código para comunicarnos a través de una red con una máquina (o entre distintos puertos dentro de la misma máquina), necesitamos generar un objeto que estaría encargado de manejar toda la información necesaria para la comunicación (hostname, dirección, puerto, etc.). Los sockets son justamente los objetos encargados de realizar esta conexión a nivel de código. Para crear sockets en Python, primero debemos importar el módulo **socket**, luego para crear un socket debemos ingresar dos argumentos, la familia de la dirección (address family) y el tipo de socket. Hay dos tipos de familias de direcciones, **AF_INET** para direcciones IPv4 y **AF_INET6** para direcciones IPv6. Con respecto a los tipos de sockets, tenemos **SOCK_STREAM** para conecciones TCP y **SOCK_DGRAM** para conexiones UDP. Ejemplo:

In [1]:
import socket

# Esto crea un socket para una conexión TCP con IPv4
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

## Arquitectura Cliente-Servidor

Esta arquitectura corresponde a un modelo de conexión entre máquinas donde algunas máquinas ofrecen un servicio (servidores) y otras máquinas(clientes) consumen estos servicios. Un cliente entonces debe conectarse a un servidor dado y usar los protocolos necesarios para obtener el servicio deseado del respectivo servidor al cual se está conectando. Un servidor por otro lado debe estar constantemente atento a potenciales conexiones de clientes, de tal forma de que cuando reciba un intento de conexión, si la conexión se establece sea capaz de entregar los servicios requeridos por el cliente. Ambas partes en la arquitectura cliente-servidor pueden aceptar conexiones del tipo TCP y UDP. La siguiente figura (extraída desde:  http://en.wikipedia.org/wiki/Client%E2%80%93server_model) muestra un diagrama de lo que sería la arquitectura cliente-servidor:

![](imgs/client-server-model.png)

### Cliente TCP en Python

  Veamos entonces cómo podríamos crear una conexión donde nuestro computador sería un **cliente TCP**:

In [ ]:
import socket
import sys


# Esto crea un socket para una conexión TCP con IPv4
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # Nos conectamos con la dirección especificada y enviamos un string 
    # (codificado en bytes) pidiendo el contenido de index.html
    # Recibimos la respuesta, el argumento indica el tamaño del bufer en bytes y 
    # finalmente aquí imprimimos los datos que recibimos después de codificarlos
    s.connect(("www.python.org", 80))
    print("GET /index.html HTTP/1.0\n\n\n".encode('ascii'))
    s.send("GET /index.html HTTP/1.0\n\n\n".encode('ascii'))  
    data = s.recv(1024)                    
    print(data, "_________")
    print(data.decode('ascii'))
    
except socket.error:
    print("No fue posible conectarse")
    sys.exit()

finally:
    # Cerramos la conexión        
    s.close()

b'GET /index.html HTTP/1.0\n\n\n'
b'HTTP/1.1 500 Domain Not Found\r\nServer: Varnish\r\nRetry-After: 0\r\ncontent-type: text/html\r\nCache-Control: private, no-cache\r\nconnection: keep-alive\r\nX-Served-By: cache-gru17120-GRU\r\nContent-Length: 221\r\nAccept-Ranges: bytes\r\nDate: Wed, 08 Nov 2017 23:49:41 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\n\r\n\n<html>\n<head>\n<title>Fastly error: unknown domain </title>\n</head>\n<body>\n<p>Fastly error: unknown domain: . Please check that this domain has been added to a service.</p>\n<p>Details: cache-gru17120-GRU</p></body></html>' _________
HTTP/1.1 500 Domain Not Found
Server: Varnish
Retry-After: 0
content-type: text/html
Cache-Control: private, no-cache
connection: keep-alive
X-Served-By: cache-gru17120-GRU
Content-Length: 221
Accept-Ranges: bytes
Date: Wed, 08 Nov 2017 23:49:41 GMT
Via: 1.1 varnish
Connection: close


<html>
<head>
<title>Fastly error: unknown domain </title>
</head>
<body>
<p>Fastly error: unknown domain: . Ple

### Servidor TCP en Python

De forma similar, si queremos implementar un **servidor TCP**:

In [ ]:
import socket

# Creamos un socket para una conexión TCP con IPv4
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()                           
port = 10001

# El metodo bind "enlaza" el socket a un puerto dado, en este caso el puerto 80
s.bind((host,port))

# Con el método listen() pedimos al sistema operativo que empiece a escuchar por
# potenciales conexiones al socket. El argumento corresponde al número máximo de 
# conexiones pendientes permitidas.
s.listen(5)

cont = 0
while True:
    # Establecemos la conexión
    socket_cliente, address = s.accept()      
    print("Obtuvimos una conexión desde %s" % str(address))
    socket_cliente.send("{}. Hola nuevo amigo!\n".format(cont).encode("ascii"))
    socket_cliente.close()
    cont += 1

En el cliente no es necesario hacer un **bind** entre el host y el puerto, ya que el sistema operativo lo hace implícitamente a través del método connect, asignando al cliente un puerto aleatorio. Sólo en casos donde el servidor al cual nos conectamos exige la dirección de cada cliente esté en un rango de puertos específicos, tendríamos que "enlazar" el cliente a un puerto específico también. En el caso del servidor el puerto debe estar enlazado con la dirección, ya que los clientes deben saber dónde ubicar exactamente al servidor para conectarse a él. El método **listen** no funciona si no enlazamos la dirección a un puerto específico.


El siguiente código correspondería a un posible cliente que se conecte al servidor implementado en la celda anterior:

In [ ]:
# Creamos un socket para una conexión TCP con IPv4
s_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Obtenemos el nombre de la máquina local
host = socket.gethostname()
port = 10001
s_cliente.connect((host,port))
data = s_cliente.recv(1024)
print(data.decode('ascii'))
s_cliente.close() 

### Cliente UDP en Python

Dado que el protocolo UDP no establece una conexión, la comunicación UDP es mucho más simple de implementar, por ejemplo para enviar un mensaje como cliente a un servidor simplemente debemos especificar la dirección y enviar el mensaje. La única consideración es que el segundo argumento al crear el socket debe ser **SOCK_DGRAM**, ejemplo:

In [2]:
import socket
MAXSIZE = 2048

#server_name = "127.0.0.1"
server_name = socket.gethostname()
server_port = 15000

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
mensaje = "Hola, simplemente te estoy enviando un mensaje".encode('ascii')
s.sendto(mensaje, (server_name, server_port))

# Opcionalmente podemos recibir información enviada de vuelta
# El metodo recvfrom() retorna además de los datos, la dirección desde 
# donde fueron enviados
data, direccion = s.recvfrom(MAXSIZE)
print(data.decode('utf-8'))

ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto

Podemos también recibir el mensaje total fragmentado, el siguiente código muestra cómo podríamos ensamblar el mensaje:

In [ ]:
fragmentos = [] 
terminado = False

while not terminado:
    chunk = s.recv(MAXSIZE)
    if not chunk:
        break
    fragmentos.append(chunk)

# Reensamblamos el mensaje final
mensage = "".join(fragmentos)


### Servidor UDP en Python

Si de forma similar queremos implementar un servidor que envía mensajes en modalidad UDP, simplemente debemos preocuparnos de responder a la misma dirección desde donde se nos ha enviado algún mensaje. Por ejemplo, el siguiente código podría representar al servidor que se comunica con el cliente implementado en la celda anterior:

In [ ]:
import socket
MAXSIZE = 2048
server_name = socket.gethostname()
server_port = 15000

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Con las doble comillas como argumento del bind indicamos que el socket es 
# alcanzable desde cualquier dirección que pueda tener el servidor. Esto 
# por supuesto también funcionaría con server_name.
s.bind(("",server_port))

while True:
    data, addr = s.recvfrom(MAXSIZE)
    respuesta = "Aquí va mi respuesta para {}".format(addr[0])
    s.sendto(respuesta.encode('utf-8'), addr)

### Envío de datos JSON

En el siguiente ejemplo veremos como generar un servidor que reciba datos y los envíe de vuelta al cliente, luego haremos un cliente que envíe datos json y los imprima una vez que el servidor los envíe de vuelta. **Haz la prueba con dos computadores, en uno ejecuta el código de la siguiente celda con la definición del servidor y en el otro la celda subsiguiente con la definición del cliente que envía los datos json**.

In [ ]:
# Implementación del servidor que recibe datos y los envía de vuelta.
# Esto cómunmente se denomina 'echo server'

import socket

host = ''        # Indicamos que es para todas las interfaces en el servidor
port = 12345     # Puerto arbitrario no-privilegiado o utilizado por otra aplicación

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(socket.gethostname())
s.bind((host, port))
s.listen(1)
conn, addr = s.accept()

print('Connected by', addr)

while True:
    data = conn.recv(1024)
    if not data: break
    conn.sendall(data)
conn.close()

In [ ]:
# Implementación del cliente que envía los datos json, 
# Poner atención en la serialización y transformación a bytes.

import socket
import sys
import json

MAX_SIZE = 1000
server_host = ""  # Aquí debe ir la dirección ip del servidor
port = 12345

# Generamos la información que enviaremos, en este caso es un simple diccinario
diction = {1: "Hola", 2: "Chao"}
message = json.dumps(diction)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    s.connect((server_host, port))

except:
    print("Error: No pudo conectarse")
    sys.exit()

# Debemos enviar bytes
s.sendall(message.encode("UTF-8"))

# Decodificamos los bytes y luego los de-serializamos con json
data = json.loads(s.recv(MAX_SIZE).decode('UTF-8'))
print(data)
s.close()

### Envío de datos con pickle

Al igual que el ejemplo anterior, podemos enviar cualquier objeto de Python serializado con pickle, el siguiente código muestra  un ejemplo de cómo conectarse al servidor anterior y enviarle datos serializados con pickle. Cuando los bytes vienen de vuelta desde el servidor lo des-serializamos y tenemos nuevamente la instancia de la clase persona que habíamos enviado:


In [ ]:
import socket
import sys
import pickle

MAX_SIZE = 1000
server_host = ""  # Debemos poner aquí la dirección ip del servidor
port = 12345


class Persona:
    
    def __init__(self, nombre, mail):
        self.nombre = nombre
        self.mail = mail


# Enviaremos esta instancia de la clase Persona
p1 = Persona("Juan Perez", "jp@algo.com")
message = pickle.dumps(p1)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    s.connect((server_host, port))
except socket.gaierror as err:
    print("Error: No pudo conectarse {}".format(err))
    sys.exit()

s.sendall(message)
data = pickle.loads(s.recv(MAX_SIZE))
print(data.nombre)
s.close()

También podríamos modificar el código del servidor para que haga alguna acción requerida con los datos que recibe y envíe el resultado de esta acción de vuelta. Por ejemplo, verificar en el servidor los datos de un cliente para iniciar sesión. **Recomendamos fuertemente conectar dos computadores y probar enviando hacia y desde ambas partes**, de tal forma de familiarizarse lo más posible con los sockets y disfrutar de los beneficios que nos ofrece la red.

## Ejemplo Práctico: Servidor con manejo de múltiples clientes en forma concurrente

En la práctica, lo más probable es que tengamos que manejar múltiples usuarios de forma concurrente en el servidor, y además enviar paquetes de datos de tamaños arbitrarios. A continuación describiremos como incorporaremos estas modificaciones.

El manejo de varios usuarios simultáneamente se logra mediante el uso de threads. Cada vez que se ejecutan los métodos `socket.accept()` y `socket.recv()` los threads dónde ellos son invocados se bloquean hasta aceptar una nueva conexión en el caso del primero, y recibir datos, en el caso del segundo. Para evitar este comportamiento deberemos crear un thread principal encargado de: 

- aceptar nuevos clientes; 
- y, cada vez que se acepte un cliente, crear un thread nuevo que se ocupe de escuchar y enviar información al nuevo cliente conectado.

Para crear un esquema de comunicación entre el cliente y el servidor que permita el intercambio de mensajes de un tamaño arbitrario deberemos modificar la cantidad de bytes que especificamos en el método `socket.recv()`. Hasta ahora hemos usado como parámetro del método `recv()` algún valor arbitrario en potencia de 2 (1024, 2048) razonable que permite el intercambio de mensajes simples. La desventaha de este esquema es que siempre vamos a limitar el tamaño de los mensajes al parámetro que recibe `socket.recv()`, lo que para algunos casos, como por ejemplo el envío de archivos, es poco práctico.

Un método simple para implementar el control de datos enviados consiste en que, antes de enviar cualquier mensaje a través del socket mediremos el largo del mensaje enviado, lo que retornará algun valor entero. Luego, calcularemos la representación en bytes de ese largo y adjuntaremos este valor al principio del mensaje enviado. Si usamos 4 bytes como encabezado de todos los mensajes para indicar el largo de éstos, podremos enviar mensajes de hasta 2^32 bytes (4 GB). Esta es una cantidad razonable para la mayoría de los casos.

Definamos a continuación una clase `Client` donde ejemplificaremos las modificaciones mencionadas:

In [ ]:
import threading
import socket


# La clase Client manejará toda la comunicación desde el lado del cliente.
# Implementa el esquema de comunicación donde los primeros 4 bytes de cada 
# mensaje indicarán el largo del mensaje enviado.

class Client:
    def __init__(self, port, host):
        print("Inicializando cliente...")

        # Inicializamos el socket principal del cliente
        self.host = host
        self.port = port
        self.socket_cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        try:
            self.connect_to_server()
            self.listen()
            self.repl()
        except:
            print("Conexión terminada")
            self.socket_cliente.close()
            exit()

    # El método connnect_to_server() creará la conexión al servidor.
    def connect_to_server(self):
        self.socket_cliente.connect((self.host, self.port))
        print("Cliente conectado exitosamente al servidor...")

    # El método listen() inicilizará el thread que escuchará los mensajes del
    # servidor. Es útil hacer un thread diferente para escuchar al servidor 
    # ya que de esa forma podremos tener comunicación asíncrona con este, es decir,
    # el servidor nos podrá enviar mensajes sin necesidad de iniciar una solicitud 
    # desde el lado del cliente.
    def listen(self):
        thread = threading.Thread(target=self.listen_thread, daemon=True)
        thread.start()


    # El método send() enviará mensajes al servidor. Implementa el mismo
    # protocolo de comunicación que mencionamos, es decir, agregar 4 bytes 
    # al principio de cada mensaje indicando el largo del mensaje enviado.
    def send(self, msg):
        msg_bytes = msg.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder="big")
        self.socket_cliente.send(msg_length + msg_bytes)


    # La función listen_thread() será lanzada como thread el cual se encarga
    # de escuchar al servidor. Vemos como se encarga de recibir 4 bytes que 
    # indicarán el largo de los mensajes. Posteriormente recibe en bloques de
    # 256 bytes el resto del mensaje hasta que éste se recibe totalmente.
    def listen_thread(self):
        while True:
            response_bytes_length = self.socket_cliente.recv(4)
            response_length = int.from_bytes(response_bytes_length, byteorder="big")
            response = b""
            
            # Recibimos datos hasta que alcancemos la totalidad de los datos 
            # indicados en los primeros 4 bytes recibidos.
            while len(response) < response_length:
                response += self.socket_cliente.recv(256)
                
            print("{}\n>>> ".format(response.decode()), end="")


    # Usaremos este método para capturar input del usuario. Lee mensajes desde 
    # el terminal y después se los pasa a `self.send()`.
    def repl(self):
        print("------ Consola ------\n>>> ", end="")
        
        while True:
            msg = input("")
            response = self.send(msg)
        
if __name__ == "__main__":
    port = 8080
    host = "0.0.0.0"

    client = Client(port, host)

Ahora implemetaremos la clase `Server()` que se encargará de controlar toda la lógica en el lado del servidor:

In [ ]:
import threading
import socket


class Server:
    
    def __init__(self, port, host):
        print("Inicializando servidor...")

        # Inicializar socket principal del servidor.
        self.host = host
        self.port = port
        self.socket_servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.bind_and_listen()
        self.accept_connections()

    # El método bind_and_listen() enlazará el socket creado con el host y puerto
    # indicado. Primero se enlaza el socket y luego que esperando por conexiones 
    # entrantes, con un máximo de 5 clientes en espera.
    def bind_and_listen(self):
        self.socket_servidor.bind((self.host, self.port))
        self.socket_servidor.listen(5)  
        print("Servidor escuchando en {}:{}...".format(self.host, self.port))
        
    # El método accept_connections() inicia el thread que aceptará clientes. 
    # Aunque podríamos aceptar clientes en el thread principal de la instancia, 
    # resulta útil hacerlo en un thread aparte que nos permitirá realizar la
    # lógica en la parte del servidor sin dejar de aceptar clientes. Por ejemplo,
    # seguir procesando archivos.
    def accept_connections(self):
        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()
        
    # El método accept_connections_thread() será arrancado como thread para 
    # aceptar clientes. Cada vez que aceptamos un nuevo cliente, iniciamos un 
    # thread nuevo encargado de manejar el socket para ese cliente.
    def accept_connections_thread(self):
        print("Servidor aceptando conexiones...")

        while True:
            client_socket, _ = self.socket_servidor.accept()
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket,),
                daemon=True
            )
            listening_client_thread.start()

    # Usaremos el método send() para enviar mensajes hacia algún socket cliente. 
    # Debemos implementar en este método el protocolo de comunicación donde los 
    # primeros 4 bytes indicarán el largo del mensaje.
    @staticmethod
    def send(value, socket):
        stringified_value = str(value)
        msg_bytes = stringified_value.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder="big")
        socket.send(msg_length + msg_bytes)


    # El método listen_client_thread() sera ejecutado como thread que escuchará a un 
    # cliente en particular. Implementa las funcionalidades del protocolo de comunicación
    # que permiten recuperar la informacion enviada.
    def listen_client_thread(self, client_socket):
        print("Servidor conectado a un nuevo cliente...")

        while True:
            response_bytes_length = client_socket.recv(4)
            response_length = int.from_bytes(response_bytes_length, byteorder="big")
            response = b""
            
            while len(response) < response_length:
                response += client_socket.recv(256)
                
            received = response.decode() 
            
            if received != "":
                # El método `self.handle_command()` debe ser definido. Este realizará 
                # toda la lógica asociado a los mensajes que llegan al servidor desde 
                # un cliente en particular. Se espera que retorne la respuesta que el 
                # servidor debe enviar hacia el cliente.
                response = self.handle_command(received, client_socket)
                self.send(response, client_socket)

                
if __name__ == "__main__":

    port = 8080
    host = "0.0.0.0"

    server = Server(port, host)

Las clases descritas en los códigos anteriores definen entidades que podrán ser usadas eficientemente a través de threads para comunicación asíncrona. Es recomendable mantener las funcionalidades de estas clases solo destinadas a funciones de networking,, i.e., solo para efectuar la comunicación entre el servidor y los clientes. Será natural que intentemos agregar código que permita manejar la lógica en el cliente o el servidor de acuerdo a los requerimientos de nuestro problema, lo que tendrá como consecuencia la generación de código muy **difícil de leer y mantener**. Por lo tanto, recomendamos fuertemente efectuar la implementación de la lógica en otras clases, similar al que usamos con interfaces gráficas.

<font size='1' face='Arial'><sup>1</sup>Agradecemos al ayudante del curso Rodolfo Palma por su colaboración para la elaboración del ejemplo práctico de este material.</font>